In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
from tracr.rasp import rasp
from tracr.compiler import basis_inference
from tracr.compiler import craft_graph_to_model
from tracr.compiler import expr_to_craft_graph
from tracr.compiler import rasp_to_graph
from tracr.craft import bases
from tracr.rasp import rasp
import networkx as nx

from rasp_generator import utils, sampling, map_primitives
from tokenizer import compile_and_tokenize, vocab

from rasp_generator.utils import FunctionWithRepr

In [2]:
sampler = sampling.ProgramSampler()
errs = sampler.sample()

In [3]:
# sop_1 = rasp.Map(lambda x: x + 1, rasp.indices)
# sel_2 = rasp.Select(sop_1, rasp.tokens, rasp.Comparison.EQ)
# sop_3 = rasp.Aggregate(sel_2, sop_1)
# sop_4 = rasp.SequenceMap(lambda x, y: x + y, sop_1, sop_3)
# sop_5 = rasp.SelectorWidth(sel_2)
# sop_6 = rasp.SequenceMap(lambda x, y: x + y, sop_3, sop_5)

sop_1 = rasp.Map(FunctionWithRepr("lambda x: x + 1"), rasp.tokens)
sel_2 = rasp.Select(sop_1, rasp.tokens, rasp.Comparison.EQ)
sop_3 = rasp.Aggregate(sel_2, sop_1)
sop_4 = rasp.SequenceMap(
    FunctionWithRepr("lambda x, y: x * y"), sop_1, sop_3)

program = sop_4

In [4]:
model, graph, sources = compile_and_tokenize.compile_rasp_to_model_and_return_graph(
    program,
    vocab={0,1,2,3,4,5},
    max_seq_len=5,
)

In [5]:
model, tokens = compile_and_tokenize.compile_and_tokenize(program)

In [6]:
tokens

{0: [],
 1: [8, 5, 0, 82, 94, 96],
 2: [27, 3, 87, 8, 94, 96, 7, 5, 4, 27, 8, 96],
 3: [9, 5, 1, 86, 8, 7, 96]}